# This code is using in Internet of Things Hackathon (Farm Connect - Soil Water Content Prediction)

It will be incluse
- Clean data
- Convert previous data for prediction next labels(consider it as time series data)
- Predict(Ensemble and fine-tune)

# 1. Install pycaret and import packages

In [ ]:
!pip install pycaret

In [ ]:
import pycaret
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
import math
from tqdm import tqdm

# 2. Get all set of train and test together

In [ ]:
DF_train_1 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/train/train/set01.csv", index_col=False)
DF_train_3 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/train/train/set03.csv", index_col=False)
DF_train_4 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/train/train/set04.csv", index_col=False)
DF_train_5 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/train/train/set05.csv", index_col=False)
DF_train_6 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/train/train/set06.csv", index_col=False)
DF_train_7 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/train/train/set07.csv", index_col=False)
DF_train_8 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/train/train/set08.csv", index_col=False)
DF_train_10 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/train/train/set10.csv", index_col=False)
DF_train_11 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/train/train/set11.csv", index_col=False)
DF_train_12 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/train/train/set12.csv", index_col=False)

DF_test_2 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/test/test/set02.csv", index_col=False)
DF_test_9 = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/test/test/set09.csv", index_col=False)

## 2.1 In Train, Del first row and create previous labels to predict next labels

In [ ]:
def create_previous_label(df): 
    labels_list = df['23SoilWC ค.ชื้นดิน']
    Labels_list = []
    for i in range(len(labels_list)):
        if i != len(labels_list)-1:
            Labels_list.append(labels_list[i])
    df=df.drop(df.index[0])
    df = df.reset_index(drop=True)
    df['previous_label'] = labels_list
    return df

DF_train_1 =  create_previous_label(DF_train_1)
DF_train_3 =  create_previous_label(DF_train_3)
DF_train_4 =  create_previous_label(DF_train_4)
DF_train_5 =  create_previous_label(DF_train_5)
DF_train_6 =  create_previous_label(DF_train_6)
DF_train_7 =  create_previous_label(DF_train_7)
DF_train_8 =  create_previous_label(DF_train_8)
DF_train_10 =  create_previous_label(DF_train_10)
DF_train_11 =  create_previous_label(DF_train_11)
DF_train_12 =  create_previous_label(DF_train_12)

## 2.2 Concat all set together

In [ ]:
DF_train = pd.concat([pd.DataFrame(DF_train_1), pd.DataFrame(DF_train_3), pd.DataFrame(DF_train_4), pd.DataFrame(DF_train_5), pd.DataFrame(DF_train_6), pd.DataFrame(DF_train_7), pd.DataFrame(DF_train_8), pd.DataFrame(DF_train_10), pd.DataFrame(DF_train_11), pd.DataFrame(DF_train_12)], ignore_index=True)

DF_test = pd.concat([pd.DataFrame(DF_test_2), pd.DataFrame(DF_test_9)], ignore_index=True)

## 2.3 Fill nan with zero

In [ ]:
DF_train = DF_train.fillna(0)
#DF_train = DF_train.dropna()
DF_test = DF_test.fillna(0)

DF_train = DF_train.reset_index(drop=True)

## 2.4 Use time column to create hour, minute and time usage

In [ ]:
H = []
M = []
TU = []
for i in range(len(DF_train)):
    T = DF_train['time'][i].split('+')[0]
    h = int(T.split(':')[0])
    m = int(T.split(':')[1])
    H.append(h)
    M.append(m)
    if int(DF_train['index'][i].split('_')[1]) == 2:
        time_start = h*60 + m -2

    
    tu = h*60 + m - time_start
    TU.append(tu)
             
DF_train['hour'] = H
DF_train['minute'] = M
DF_train['time_use'] = TU
                 
DF_train = DF_train.drop(columns=['index', 'time'])


In [ ]:
DF_train

In [ ]:
H = []
M = []
TU = []
for i in range(len(DF_test)):
    T = DF_test['time'][i].split('+')[0]
    h = int(T.split(':')[0])
    m = int(T.split(':')[1])
    H.append(h)
    M.append(m)
    if int(DF_test['index'][i].split('_')[1]) == 2:
        time_start = h*60 + m -2

    
    tu = h*60 + m - time_start
    TU.append(tu)
    
    
    
             
DF_test['hour'] = H
DF_test['minute'] = M
DF_test['time_use'] = TU
                 
DF_test = DF_test.drop(columns=['index', 'time'])

In [ ]:
DF_test

# 3. Train with pycaret regression

## 3.1 Setup pycaret.regression

In [ ]:
# Setup models for regression problem
from pycaret.regression import *
# from pycaret.classification import *

regression_setup = pycaret.regression.setup(data=DF_train,
                                                train_size=0.80,
                                                target='23SoilWC ค.ชื้นดิน',
                                                session_id=122,
                                                imputation_type='iterative',
                                                fold_shuffle=True,
                                                remove_outliers=True,
                                                normalize = True,
                                                )

## 3.2 Train models

In [ ]:
best = compare_models(n_select = 5, sort = 'MAE')

## 3.3 Ensembl models

In [ ]:
# ensembl =  ensemble_model(best[0], method = 'Boosting')

## 3.4 Fine-tune models

In [ ]:
# tune_our_model = tune_model(ensembl, optimize='MAE')
# f_model = finalize_model(tune_our_model)

# 4. Predict

In [ ]:
Submit = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/sample_submission.csv")
Submit

In [ ]:
index_list = Submit['index']

DF_test['index'] = Submit['index']
DF_test

## 4.1 Predict row by row (This model need previous prediction, so i use given label for the first row and use those data in next prediction)

In [ ]:
## The accuracy of models does not increase after ensemble and fine-tune model, so I use the first best model (best[0])

Ans = []
for i in tqdm(range(len(DF_test))):
    if i == 0:
        last_label = 31.1
        Ans.append(last_label)
    elif i == 113:
        last_label = 32.7
        Ans.append(last_label)
    else:
        select_row = DF_test.loc[DF_test['index'] == index_list[i]]
        select_row['previous_label'] = last_label
        select_row = select_row.drop('index', axis=1)
        prediction = predict_model(best[0], data = select_row)
        last_label = float(prediction['prediction_label'])
        Ans.append(last_label)


In [ ]:
# Plot the prediction

plt.plot(Ans)
plt.show()

## 4.2 Create and save submission file

In [ ]:
Submit = pd.read_csv("/kaggle/input/farm-connect-soil-water-content-prediction/sample_submission.csv")
Submit['23SoilWC ค.ชื้นดิน'] = Ans

Submit.loc[0,'23SoilWC ค.ชื้นดิน'] = 31.1
Submit.loc[113,'23SoilWC ค.ชื้นดิน'] = 32.7

Submit

In [ ]:
Submit.to_csv('/kaggle/working/submission.csv', index=False)